In [ ]:
from quantopian.research import prices, symbols,returns
from cvxopt import matrix, solvers
import cvxopt as opt
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
import pandas as pd

period_start = '2013-03-31'
period_end = '2018-03-31'

#returns
aapl_returns = returns(
    assets=symbols('AAPL'),
    start=period_start,
    end=period_end,
)
amazon_returns = returns(
    assets=symbols('AMZN'),
    start=period_start,
    end=period_end,
)
ibm_returns = returns(
    assets=symbols('IBM'),
    start=period_start,
    end=period_end,
)
microsoft_returns = returns(
    assets=symbols('MSFT'),
    start=period_start,
    end=period_end,
)
# Combine results into a pandas DataFrame and plot

df_return=pd.DataFrame({   
    'AAPL_return': aapl_returns,
    'AMZN_return': amazon_returns,
    'IBM_return': ibm_returns,
    'MSFT_return':microsoft_returns
})
df_return

mu=df_return.mean()
mu
sigma=matrix(np.matrix(df_return.cov()))
m=matrix(mu)

# min var(20000$) due to the a large amount of money, we need to choose porfolio with low Variance
P=2*sigma
q=matrix(np.zeros((4,1)))
G=-matrix(np.identity((4)))
h=matrix(np.zeros(4))
A = matrix(np.ones((1,4)))
b = matrix(1.0)
#solve qp
sol = solvers.qp(P, q, G,h,A, b) 
w=sol['x']
print(w)

# mean-var(5000)
rd = np.mean(m)
P = 2 * sigma
q = matrix(np.zeros((4, 1)))
G = matrix(np.concatenate((-np.array(m.T),-np.identity(4)), 0))            
h = matrix(np.concatenate((-np.ones((1,1))*rd,-np.zeros((4,1))), 0))              
# equality constraints
A = matrix(np.ones((1,4)))
b = matrix(1.0)
sol = solvers.qp(P, q, G,h,A, b) 
w=sol['x']
print(w)

#max return(1000) due to the a large amount of money, we can choose the stock with highest return
P=matrix(np.zeros([4,4]))
q=matrix(-mu)
G=-matrix(np.identity((4)))
h=matrix(np.zeros(4))
A = matrix(np.ones((1,4)))
b = matrix(1.0)
#solve qp
sol = solvers.qp(P, q, G,h,A, b) 
w=sol['x']
print(w)